In [7]:
import ee
import geemap
import os

ee.Authenticate()
ee.Initialize()

In [ ]:
output_dir = os.path.expanduser('~/Desktop/Thesis/TrainingData')
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# ==========================================
# 2. LOAD YOUR SHAPEFILE
# ==========================================

# Load the heavy shapefile directly from the server

ee_object = ee.FeatureCollection("projects/integrated-hawk-485001-k3/assets/PH_shapefile")

roi = ee_object.geometry()

Map = geemap.Map()
Map.centerObject(roi, 12)
Map.addLayer(roi, {'color': 'red'}, 'My Shapefile ROI')
Map  # Uncomment to visualize

Complex ROI loaded successfully from Assets!


Map(center=[11.69262582935914, 122.90080052313625], controls=(WidgetControl(options=['position', 'transparent_…

In [12]:
# ==========================================
# 3. PREPARE SENTINEL-2 IMAGE (Same as before)
# ==========================================
def mask_s2_clouds(image):
    qa = image.select('QA60')
    mask = qa.bitwiseAnd(1 << 10).eq(0).And(qa.bitwiseAnd(1 << 11).eq(0))
    return image.updateMask(mask).divide(10000)

collection = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterDate('2023-01-01', '2023-12-31')
    .filterBounds(roi)  # Now filtering by your shapefile
    .map(mask_s2_clouds))

image = collection.median().clip(roi)
cnn_image = image.select(['B4', 'B3', 'B2', 'B8'])

In [ ]:
# ==========================================
# 4. DOWNLOAD CHIPS FROM SHAPEFILE
# ==========================================
output_dir = os.path.expanduser('~/User/Desktop/Thesis/TrainingData/Sentinel2')
SCALE = 10  # Sentinel-2 resolution in meters

print("Generating fishnet grid over your shapefile...")

# Generate a grid (fishnet) specifically bounded by your shapefile.
# This ensures you only download tiles inside your specific polygon.
fishnet = geemap.fishnet(ee_object, h_interval=0.02, v_interval=0.02, delta=0.02)

print("Downloading chips...")
geemap.download_ee_image_tiles_parallel(
    image=cnn_image,
    output_dir=output_dir,
    features=fishnet,
    scale=SCALE, 
    crs='EPSG:3857',
    prefix='s2_shp_'
)

Generating fishnet grid over your shapefile...


KeyboardInterrupt: 